# Building a Neural Network
In this demo we are going to demonstrate how to build and train a model using PyTorch.

This model will be a neural network type.

# Create a Class
REVIEW: Building a neural network is made simple in PyTorch.

This is because of the `nn.Module` which we inherit when we create our class to simplify building, managing and organizing our model.

This is used to lay the blueprint for our model.

### Structure of our Class

`__init__()`: This is where we define the layers of our network.


`forward()`: This is where we define how data is processed through our layers to get a prediction.


In [2]:
# Import the nn module
import torch.nn as nn 

In [3]:
# Create a simple class
class SimpleNeuralNetwork(nn.Module):
    def __init__(self):
        super(SimpleNeuralNetwork, self).__init__() # initialize superclass for automatic parameters
        
        # Define the layers: an input layer, a hidden layer, and an output layer
        self.input_layer = nn.Linear(10, 20)  # Input size of 10, output size of 20
        self.hidden_layer = nn.Linear(20, 15) # Hidden layer with input size of 20, output size of 15
        self.output_layer = nn.Linear(15, 1)  # Output layer with input size of 15, output size of 1
        
        # Define the activation function (introduces non-linearity into the model)
        self.activation = nn.ReLU()

    # Define the forward pass
    def forward(self, x):
        x = self.activation(self.input_layer(x))  # Pass data through the input layer
        x = self.activation(self.hidden_layer(x)) # Pass data through the hidden layer
        x = self.output_layer(x)                  # Pass data through the output layer (no activation here)
        return x

In [ ]:
# Demonstrate our model with an example
import torch

# Create a tensor with shape (5, 10) - batch of 5 samples, each with 10 features
example_tensor = torch.randn(5, 10)
print(example_tensor.size())


In [ ]:
# Create the input layer as it is in our Class
input_layer = nn.Linear(10, 20)

# Run the example through our input layer
input_linear_example = input_layer(example_tensor)
print(input_linear_example.size())

In [ ]:
# Do the same but with the hidden layer
hidden_layer = nn.Linear(20, 15)

# Run the input_linear_example through hidden layer
hidden_linear_example = hidden_layer(input_linear_example)
print(hidden_linear_example.size())

In [ ]:
# Same for the output layer
output_layer = nn.Linear(15, 1)

# Run hidden_linear_example through output layer
ouput_linear_example = output_layer(hidden_linear_example)
print(ouput_linear_example.size())

In [ ]:
# Now with activation layer ReLU befor and after on the output example
print(f"Before ReLU: {ouput_linear_example}\n\n")

In [ ]:
# Run through ReLU
activation_relu_example = nn.ReLU()(ouput_linear_example)
print(f"After ReLU: {activation_relu_example}")

In [ ]:
# Create an example with forward

# Recreate instance of activation layer
activation = nn.ReLU()

# Pass example through input layer and apply ReLU
x = activation(input_layer(example_tensor))
# Pass through hidden layer and apply ReLU 
x = activation(hidden_layer(x))
# Pass through output layer (no activation)
output = output_layer(x)

print("Example Tensor:")
print(example_tensor)
print("\nOutput Tensor:")
print(output)

# Create the Model


In [33]:
# Initialize the model
model = SimpleNeuralNetwork()


In [ ]:
# Show the layers
print(model)

# Model Parameters
Layers have associate weights and biases.

These weights and biases get adjusted during model training.

Lucky for us, the adjustments are tracked automatically by PyTorch!

In [ ]:
# Loop through the parameters in human readable
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

In [ ]:
# here we see the current shape and values for each layers weight and bias

In [ ]:
# Another way to display parameters
for param in model.parameters():
    print(param)

#### Review Autograd
In PyTorch, autograd automatically computes gradients, which is essential for training a neural network by adjusting its weights to improve predictions.

`model.parameters()` provides access to the model’s weights and biases, which are PyTorch tensors that have `requires_grad=True`. 

This means they automatically participate in PyTorch's autograd system, which tracks operations on these tensors to build a computation graph.

Very Powerful!

# Define a Loss Function and Optimizer

REVIEW:

Loss Function: Measures how well the model's predictions match the actual data, guiding the model on how much to adjust to improve.


Optimizer: Updates the model's parameters based on the loss, using methods like gradient descent to minimize errors and improve performance over time.

In [39]:
# Create a common loss function for an Image Classifier

# Part of the nn module
import torch.nn as nn

In [41]:
# Create an instance of the loss function
criterion = nn.CrossEntropyLoss() # commonly used for classification 

In [42]:
# Import optimizer modules
import torch.optim as optim

In [57]:
# Create an optimizer instance and provide it the parameters
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # SGD is commonly used in classifcation

# Go through the steps of Training a Model

But first we must create our data and transformations.

We are going to use the MNIST preloaded Dataset.

In [1]:
import torchvision.datasets
import torch
import torchvision
from torchvision.transforms import v2

# Define Transforms. Already resized.
transform = v2.Compose(
    [v2.ToImage(), 
     v2.ToDtype(torch.float32, scale=True),
     v2.Normalize((0.5,), (0.5,))]) # These are grayscale images

# Training dataset and dataloader
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                          shuffle=True, num_workers=1)

# Validation dataset and dataloader
val_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32,
                                         shuffle=False, num_workers=1)


# Create a NN for an Image Classifier
Here we are going to create a Neural Network to train an image classifcation model.



In [2]:
# Create the Class
import torch.nn as nn
# This module simplifies a way to import Operations (Activation Functions)
import torch.nn.functional as F 


class ImageClassificationNet(nn.Module):
    def __init__(self):
        super(ImageClassificationNet, self).__init__()
        # Takes an input with 1 channel , outputs 6 feature maps, uses a 5x5 kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        # Takes 6 input feature maps from the previous layer, outputs 16 feature maps, uses a 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5)
        # Define a max pooling layer to downsample the feature maps by a factor of 2
        self.pool = nn.MaxPool2d(2, 2)
        # Takes the flattened output from the convolutional layers (16 feature maps of size 5x5) and outputs 120 units
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        # Define the second fully connected layer, which maps 120 units to 84 units
        self.fc2 = nn.Linear(120, 84)
        # 10 classes for classification
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Pass input `x` through the first convolutional layer, apply ReLU activation, then apply max pooling
        x = self.pool(F.relu(self.conv1(x)))
        # Pass the result through the second convolutional layer, apply ReLU activation, then apply max pooling
        x = self.pool(F.relu(self.conv2(x)))
        # Flatten the feature maps into a 1D vector, keeping the batch dimension
        x = torch.flatten(x, 1)
        # Pass through the first fully connected layer and apply ReLU activation
        x = F.relu(self.fc1(x))
        # Pass through the second fully connected layer and apply ReLU activation
        x = F.relu(self.fc2(x))
        # Pass through the third fully connected layer to get the output (raw scores for each class)
        x = self.fc3(x)
        # Return the output scores (logits) for each class
        return x


# Create the Model

In [3]:
# Remember how to check for GPU?
import torch

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Creates an instance of our model
model = ImageClassificationNet().to(device)

# Print it
print(model)

# Loss Funtion and Optimizer
Use same as above

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Create a Training Loop
A training loop in PyTorch is the process of iteratively feeding data through a model, calculating the loss, and updating the model’s parameters to minimize that loss. 

This loop continues for a set number of epochs or until the model reaches satisfactory performance on the training data.

In [6]:
# Define our number of training loops
N_EPOCHS = 3

In [ ]:
for epoch in range(N_EPOCHS):  # Loop over the dataset N_EPOCH times
    
    running_loss = 0.0  # Initialize the running loss for the current epoch
    
    # Loop over the training data in batches
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data  # Unpack the data; inputs are the images, labels are the classes
        
        optimizer.zero_grad()  # Clear the gradients for the optimizer to avoid accumulation from previous steps

        outputs = model(inputs)  # Forward pass: compute the model's predictions on the inputs
        loss = criterion(outputs, labels)  # Calculate the loss by comparing predictions to true labels
        loss.backward()  # Backward pass: compute gradients of the loss with respect to model parameters
        optimizer.step()  # Update model parameters based on the computed gradients
        
        running_loss += loss.item()  # Accumulate the loss for the current epoch

    # Print the average loss for this epoch by dividing the accumulated loss by the number of batches
    print(f"Epoch: {epoch} Loss: {running_loss/len(train_loader)}")


# Create a Training Loop with Validation

In [ ]:
for epoch in range(N_EPOCHS):  # Loop over the dataset N_EPOCH times
    
    ####### TRAINING
    training_loss = 0.0  # Initialize the training loss for the current epoch
    # Set the model to training mode
    model.train()
    # Loop over the training data in batches
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data  # Unpack the data; inputs are the images, labels are the classes
        
        optimizer.zero_grad()  # Clear the gradients for the optimizer to avoid accumulation from previous steps

        outputs = model(inputs)  # Forward pass: compute the model's predictions on the inputs
        loss = criterion(outputs, labels)  # Calculate the loss by comparing predictions to true labels
        loss.backward()  # Backward pass: compute gradients of the loss with respect to model parameters
        optimizer.step()  # Update model parameters based on the computed gradients
        
        training_loss += loss.item()  # Accumulate the training loss for the current epoch

    ######## VALIDATION
    val_loss = 0.0 # Initialize the validation loss for the current epoch
    # Set the model to evaluation 
    model.eval()

    # Loop over the validation data in batches
    for i, data in enumerate(val_loader, 0):
        inputs, labels = data  # Unpack the data like we do above
        
        outputs = model(inputs)  # Compute predictions
        loss = criterion(outputs, labels)  # Calculate the loss by
        
        #### NOTICE we do not compute gradients and/or adjust weights #### 
        val_loss += loss.item()  # Accumulate the loss for the current epoch

    # Print the training loss and the val loss
    print(f"Epoch: {epoch} Train Loss: {training_loss/len(train_loader)} Val Loss: {val_loss/len(val_loader)}")

# About Loss
If validation continues to decrease, its performing well.

If training continues to decrease but validation does not, its likely that its overfitting.

Likely we would need many more epochs to train an accurate model.